https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

# Create the lagged features 24 +1
lags = 48 + 1

# Create the lagged features
for i in range(1, lags):
    data['lag' + str(i)] = data['count'].shift(i)

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

print(cutoffs)

# Define cutoffs for cross validation with 5 folds every two months with first cut on 2022-06-01, last day 2023-01-15 and horizon of 15 days
#cutoffs = pd.to_datetime(['2022-06-01', '2022-08-01', '2022-10-01', '2022-12-01', '2023-01-15'])
# print(cutoffs)


0        2022-04-02 00:00:00
1        2022-04-02 00:30:00
2        2022-04-02 01:00:00
3        2022-04-02 01:30:00
4        2022-04-02 02:00:00
                ...         
17467    2023-03-31 21:30:00
17468    2023-03-31 22:00:00
17469    2023-03-31 22:30:00
17470    2023-03-31 23:00:00
17471    2023-03-31 23:30:00
Name: ds, Length: 17472, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 2909 2910 2911]
  Test:  index=[2912 2913 2914 ... 5821 5822 5823]
Fold 1:
  Train: index=[   0    1    2 ... 5821 5822 5823]
  Test:  index=[5824 5825 5826 ... 8733 8734 8735]
Fold 2:
  Train: index=[   0    1    2 ... 8733 8734 8735]
  Test:  index=[ 8736  8737  8738 ... 11645 11646 11647]
Fold 3:
  Train: index=[    0     1     2 ... 11645 11646 11647]
  Test:  index=[11648 11649 11650 ... 14557 14558 14559]
Fold 4:
  Train: index=[    0     1     2 ... 14557 14558 14559]
  Test:  index=[14560 14561 14562 ... 17469 17470 17471]
['2022-06-01 15:30:00', '2022-08-01 07:30:00', '2022-09-30 23:

In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    train = pd.DataFrame(train_data)
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
     # add regressors with lags 1 to 24
    for i in range(1,49):
        train_model.add_regressor('lag'+str(i), standardize=False)
        train['lag'+str(i)] = train['y'].shift(i)


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")

     # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    


    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


16:38:24 - cmdstanpy - INFO - Chain [1] start processing
16:38:26 - cmdstanpy - INFO - Chain [1] done processing
16:38:27 - cmdstanpy - INFO - Chain [1] start processing
16:38:28 - cmdstanpy - INFO - Chain [1] done processing
16:38:28 - cmdstanpy - INFO - Chain [1] start processing
16:38:28 - cmdstanpy - INFO - Chain [1] start processing
16:38:28 - cmdstanpy - INFO - Chain [1] done processing
16:38:29 - cmdstanpy - INFO - Chain [1] start processing
16:38:29 - cmdstanpy - INFO - Chain [1] start processing
16:38:30 - cmdstanpy - INFO - Chain [1] done processing
16:38:30 - cmdstanpy - INFO - Chain [1] done processing
16:38:31 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


16:38:32 - cmdstanpy - INFO - Chain [1] start processing
16:38:35 - cmdstanpy - INFO - Chain [1] done processing
16:38:36 - cmdstanpy - INFO - Chain [1] start processing
16:38:36 - cmdstanpy - INFO - Chain [1] start processing
16:38:36 - cmdstanpy - INFO - Chain [1] done processing
16:38:37 - cmdstanpy - INFO - Chain [1] start processing
16:38:37 - cmdstanpy - INFO - Chain [1] start processing
16:38:37 - cmdstanpy - INFO - Chain [1] done processing
16:38:38 - cmdstanpy - INFO - Chain [1] start processing
16:38:38 - cmdstanpy - INFO - Chain [1] done processing
16:38:39 - cmdstanpy - INFO - Chain [1] done processing
16:38:40 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 5.01}


17:40:32 - cmdstanpy - INFO - Chain [1] start processing
17:40:35 - cmdstanpy - INFO - Chain [1] done processing
17:40:37 - cmdstanpy - INFO - Chain [1] start processing
17:40:37 - cmdstanpy - INFO - Chain [1] start processing
17:40:38 - cmdstanpy - INFO - Chain [1] start processing
17:40:38 - cmdstanpy - INFO - Chain [1] start processing
17:40:38 - cmdstanpy - INFO - Chain [1] start processing
17:40:39 - cmdstanpy - INFO - Chain [1] done processing
17:40:40 - cmdstanpy - INFO - Chain [1] done processing
17:40:42 - cmdstanpy - INFO - Chain [1] done processing
17:40:44 - cmdstanpy - INFO - Chain [1] done processing
17:40:44 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


17:40:46 - cmdstanpy - INFO - Chain [1] start processing
17:40:49 - cmdstanpy - INFO - Chain [1] done processing
17:40:51 - cmdstanpy - INFO - Chain [1] start processing
17:40:51 - cmdstanpy - INFO - Chain [1] start processing
17:40:51 - cmdstanpy - INFO - Chain [1] start processing
17:40:51 - cmdstanpy - INFO - Chain [1] start processing
17:40:52 - cmdstanpy - INFO - Chain [1] done processing
17:40:52 - cmdstanpy - INFO - Chain [1] start processing
17:40:54 - cmdstanpy - INFO - Chain [1] done processing
17:40:55 - cmdstanpy - INFO - Chain [1] done processing
17:40:57 - cmdstanpy - INFO - Chain [1] done processing
17:40:58 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  0.813111  
1                       2.51  0.798228  
2                       

In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 0.01, 'seasonality_mode': 'additive', 'holidays_prior_scale': 2.51, 'changepoint_prior_scale': 0.001}
Best MASE 0.7811516414461035
[0.8131114652501816, 0.7982278548382845, 0.7983992519782875, 0.7985702959343615, 0.7856899451828758, 0.7881483803927852, 0.7913283134852805, 0.7844045682892689, 0.8030778302609464, 0.7898046498882915, 0.7904386408206738, 0.7908856781016333, 0.7811516414461035, 0.7859270340664377, 0.7855038318682381, 0.7846711513661818, 0.804112389271182, 0.7921428279902043, 0.7912026808833313, 0.7918939379782839, 0.784204604808554, 0.7865739542423688, 0.7872159406126565, 0.789942692639467, 0.8036796621380558, 0.7896694471410104, 0.7925299433644752, 0.7907417733102473, 0.7837340927635361, 0.78778797801961, 0.7856271034455996, 0.7878390939451004, 0.8287136036066283, 0.8035116518423019, 0.803746363757486, 0.8035400949744066, 0.7852704940659275, 0.7871753995150286, 0.786596934491173, 0.7867831354818592, 0.8146416583273403, 0.793580535